In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

<b>The goal of this workbook is to pickup from where the 'Business objectives based model selection' worksheet left off, and further tune the selected model to maximize business benefit</b>

<b>First we read the predictions output from the model and also include raw probabilities - 'P (Churn=0)' and 'P (Churn=1)'</b>

In [100]:
df_gbm = pd.read_csv('..\\..\\Data\\GBM_predictions.csv')
df_gbm.head()

,VMail Message,CustServ Calls,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge,City,Account Length,Churn,Int'l Plan,VMail Plan,P (Churn=0),P (Churn=1),Prediction (Churn)
0,-0.591671,1.092477,-0.619642,-0.425761,-1.260592,0.920150,0.761030,-0.171066,0.211502,0.736852,Cedarville,-0.152297,0,0,0,9.941570e-01,0.005843,0
1,-0.591671,1.092477,1.024678,0.591580,0.747175,-3.348331,-1.129754,1.828167,2.243020,0.842984,Lomita,1.781302,0,0,0,9.996630e-01,0.000337,0
2,1.599971,-1.188040,1.373474,-0.614758,-1.009622,-0.759404,0.454416,-1.414545,0.617806,1.532848,Lomita,-1.633887,0,0,1,9.993446e-01,0.000655,0
3,1.819135,1.092477,0.277260,0.442542,-0.055932,-0.262962,0.352211,-1.959391,0.617806,0.100055,Fresno,0.877281,0,1,1,2.004790e-51,1.000000,1
4,-0.591671,1.092477,0.077949,-0.981951,-0.758651,-0.673571,1.783075,-0.258945,-1.007408,-0.297943,Hamilton City,-0.076962,0,0,0,9.930186e-01,0.006981,0


<b>Creating prediction columns for each threshold interval of 0.1 in range [0,1]</b>

In [104]:
idx = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
for i in idx:
    df_gbm['Pred_' + str(i)] =  np.where(df_sub['P (Churn=1)'] >= i, 1, 0) 
df_gbm.head()

,VMail Message,CustServ Calls,Day Calls,Day Charge,Eve Calls,Eve Charge,Night Calls,Night Charge,Intl Calls,Intl Charge,...,FN0.8,TP0.8,TN0.9,FP0.9,FN0.9,TP0.9,TN1,FP1,FN1,TP1
0,-0.591671,1.092477,-0.619642,-0.425761,-1.260592,0.920150,0.761030,-0.171066,0.211502,0.736852,...,0,0,1,0,0,0,1,0,0,0
1,-0.591671,1.092477,1.024678,0.591580,0.747175,-3.348331,-1.129754,1.828167,2.243020,0.842984,...,0,0,1,0,0,0,1,0,0,0
2,1.599971,-1.188040,1.373474,-0.614758,-1.009622,-0.759404,0.454416,-1.414545,0.617806,1.532848,...,0,0,1,0,0,0,1,0,0,0
3,1.819135,1.092477,0.277260,0.442542,-0.055932,-0.262962,0.352211,-1.959391,0.617806,0.100055,...,0,0,0,1,0,0,0,1,0,0
4,-0.591671,1.092477,0.077949,-0.981951,-0.758651,-0.673571,1.783075,-0.258945,-1.007408,-0.297943,...,0,0,1,0,0,0,1,0,0,0


<b>Calculating truth table values for each threshold</b>

In [103]:
for i in idx:
    df_gbm['TN' + str(i)] = np.where((df_gbm['Pred_' + str(i)] == 0) & (df_gbm['Churn'] == 0), 1, 0)
    df_gbm['FP' + str(i)] = np.where((df_gbm['Pred_' + str(i)] == 1) & (df_gbm['Churn'] == 0), 1, 0)
    df_gbm['FN' + str(i)] = np.where((df_gbm['Pred_' + str(i)] == 0) & (df_gbm['Churn'] == 1), 1, 0)
    df_gbm['TP' + str(i)] = np.where((df_gbm['Pred_' + str(i)] == 1) & (df_gbm['Churn'] == 1), 1, 0)

<b>Getting a smaller subset (columns) of dataset</b>

In [67]:
col_list = ['Churn','P (Churn=1)']
for i in idx:
    col_list.append('TN' + str(i))
    col_list.append('FP' + str(i))
    col_list.append('FN' + str(i))
    col_list.append('TP' + str(i))
df_sub = df_gbm[col_list]

<b>Aggregating across all rows</b><br/><b>Defining business objectives/constraints and function to be able to integrate them with truth table values</b>


In [107]:
df_cf = df_sub.sum(axis = 0)

tp_bnft = 1500
fn_cost = -2500
fp_cost = -200
tn_bnft = 100

def compute_benefit (tp, fn, fp, tn) -> float:
    return 0.8*tp*tp_bnft + (1-0.8)*tp*fn_cost + fn*fn_cost + tn*tn_bnft + 0.90*fp*fp_cost + (1-0.9)*fp*tn_bnft

<b>Running the function defined above for each threshold and storing net-benefit values for each</b>

In [112]:
df_res = pd.DataFrame(index = idx, columns=['benefit'])
print ('i   tp   fn   fp   tn    benefit')
for i in idx:
    tp =  df_cf['TP' + str(i)]
    fn =  df_cf['FN' + str(i)]
    fp =  df_cf['FP' + str(i)]
    tn =  df_cf['TN' + str(i)]
    df_res.at[i,'benefit'] = compute_benefit(tp, fn, fp, tn)
    print (float(i), tp, fn, fp, tn, round(df_res.get_value(i,'benefit')))

i   tp   fn   fp   tn    benefit
0.0 97.0 0.0 570.0 0.0 -29000.0
0.1 80.0 17.0 82.0 488.0 48360.0
0.2 80.0 17.0 72.0 498.0 51060.0
0.3 78.0 19.0 63.0 507.0 47090.0
0.4 76.0 21.0 55.0 515.0 42850.0
0.5 72.0 25.0 51.0 519.0 31130.0
0.6 66.0 31.0 50.0 520.0 12200.0
0.7 60.0 37.0 46.0 524.0 -5920.0
0.8 54.0 43.0 44.0 526.0 -24580.0
0.9 49.0 48.0 42.0 528.0 -40040.0
1.0 31.0 66.0 40.0 530.0 -97100.0


C:\Software\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


<b>Final table with probability cut-off threshold and associated business (net) benefit if we were to choose that threshold</b><br/><br/>
<b>NOTE: We could have chosen the threshold using an ROC curve, but this method takes it a step further by integrating business objectives into it</b>

In [110]:
df_res
# quick sanity check: the value at 0.5 below should agree with the 'baseline' benefit computed using the excel document...
# which uses predictions (intead of probabilities) from the model output

,benefit
0.0,-29000
0.1,48360
0.2,51060
0.3,47090
0.4,42850
0.5,31130
0.6,12200
0.7,-5920
0.8,-24580
0.9,-40040


<b>We see that the net benefit peaks at a threshold of 0.2 as we go from 0 to 1, thus choosing this cut-off value will maximize profits for the marketing campaign</b>

In [ ]:
#end of workbook